In [ ]:
import torch
import torch.nn as nn
import time
import argparse

import os
import datetime
import torch.nn.functional as F
import random

from torch.distributions.categorical import Categorical
import math
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable
from torch.optim import lr_scheduler
import matplotlib.pyplot as plt
import time
from tqdm import tqdm_notebook
from tqdm import tqdm_notebook
import math
import numpy as np
import torch
import tqdm
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset
import matplotlib.pyplot as plt
from scipy.spatial import distance
# visualization 
%matplotlib inline
from IPython.display import set_matplotlib_formats, clear_output
import matplotlib_inline
matplotlib_inline.backend_inline.set_matplotlib_formats('png2x','pdf')
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings("ignore", category=UserWarning)

device = torch.device("cpu"); gpu_id = -1 # select CPU

gpu_id = '0' # select a single GPU  
#gpu_id = '2,3' # select multiple GPUs  
os.environ["CUDA_VISIBLE_DEVICES"] = str(gpu_id)  
if torch.cuda.is_available():
    device = torch.device("cuda")
    print('GPU name: {:s}, gpu_id: {:s}'.format(torch.cuda.get_device_name(0),gpu_id))   
    
print(device)

# HPN Large

In [ ]:
def compute_tour_length(x, tour): 
    """
    Compute the length of a batch of tours
    Inputs : x of size (bsz, nb_nodes, 2) batch of tsp tour instances
             tour of size (bsz, nb_nodes) batch of sequences (node indices) of tsp tours
    Output : L of size (bsz,)             batch of lengths of each tsp tour
    """
    bsz = x.shape[0]
    nb_nodes = x.shape[1]
    arange_vec = torch.arange(bsz, device=x.device)
    first_cities = x[arange_vec, tour[:,0], :] # size(first_cities)=(bsz,2)
    previous_cities = first_cities
    L = torch.zeros(bsz, device=x.device)
    with torch.no_grad():
        for i in range(1,nb_nodes):
            current_cities = x[arange_vec, tour[:,i], :] 
            L += torch.sum( (current_cities - previous_cities)**2 , dim=1 )**0.5 # dist(current, previous node) 
            previous_cities = current_cities
        L += torch.sum((current_cities - first_cities)**2 , dim=1)**0.5 # dist(last, first node)  
    return L

class TransEncoderNet(nn.Module):
    """
    Encoder network based on self-attention transformer
    Inputs :  
      h of size      (bsz, nb_nodes+1, dim_emb)    batch of input cities
    Outputs :  
      h of size      (bsz, nb_nodes+1, dim_emb)    batch of encoded cities
      score of size  (bsz, nb_nodes+1, nb_nodes+1) batch of attention scores
    """
    
    def __init__(self, nb_layers, dim_emb, nb_heads, dim_ff, batchnorm):
        super(TransEncoderNet, self).__init__()
        assert dim_emb == nb_heads* (dim_emb//nb_heads) # check if dim_emb is divisible by nb_heads
        self.MHA_layers = nn.ModuleList( [nn.MultiheadAttention(dim_emb, nb_heads) for _ in range(nb_layers)] )
        self.linear1_layers = nn.ModuleList( [nn.Linear(dim_emb, dim_ff) for _ in range(nb_layers)] )
        self.linear2_layers = nn.ModuleList( [nn.Linear(dim_ff, dim_emb) for _ in range(nb_layers)] )   
        if batchnorm:
            self.norm1_layers = nn.ModuleList( [nn.BatchNorm1d(dim_emb) for _ in range(nb_layers)] )
            self.norm2_layers = nn.ModuleList( [nn.BatchNorm1d(dim_emb) for _ in range(nb_layers)] )
        else:
            self.norm1_layers = nn.ModuleList( [nn.LayerNorm(dim_emb) for _ in range(nb_layers)] )
            self.norm2_layers = nn.ModuleList( [nn.LayerNorm(dim_emb) for _ in range(nb_layers)] )
        self.nb_layers = nb_layers
        self.nb_heads = nb_heads
        self.batchnorm = batchnorm
        
    def forward(self, h):      
        # PyTorch nn.MultiheadAttention requires input size (seq_len, bsz, dim_emb) 
        h = h.transpose(0,1) # size(h)=(nb_nodes, bsz, dim_emb)  
        # L layers
        for i in range(self.nb_layers):
            h_rc = h # residual connection, size(h_rc)=(nb_nodes, bsz, dim_emb)
            h, score = self.MHA_layers[i](h, h, h) # size(h)=(nb_nodes, bsz, dim_emb), size(score)=(bsz, nb_nodes, nb_nodes)
            # add residual connection
            
            h = h_rc + h # size(h)=(nb_nodes, bsz, dim_emb)
            if self.batchnorm:
                # Pytorch nn.BatchNorm1d requires input size (bsz, dim, seq_len)
                h = h.permute(1,2,0).contiguous() # size(h)=(bsz, dim_emb, nb_nodes)
                h = self.norm1_layers[i](h)       # size(h)=(bsz, dim_emb, nb_nodes)
                h = h.permute(2,0,1).contiguous() # size(h)=(nb_nodes, bsz, dim_emb)
            else:
                h = self.norm1_layers[i](h)       # size(h)=(nb_nodes, bsz, dim_emb) 
            # feedforward
            h_rc = h # residual connection
            h = self.linear2_layers[i](torch.relu(self.linear1_layers[i](h)))
            h = h_rc + h # size(h)=(nb_nodes, bsz, dim_emb)
            if self.batchnorm:
                h = h.permute(1,2,0).contiguous() # size(h)=(bsz, dim_emb, nb_nodes)
                h = self.norm2_layers[i](h)       # size(h)=(bsz, dim_emb, nb_nodes)
                h = h.permute(2,0,1).contiguous() # size(h)=(nb_nodes, bsz, dim_emb)
            else:
                h = self.norm2_layers[i](h) # size(h)=(nb_nodes, bsz, dim_emb)
        # Transpose h
        h = h.transpose(0,1) # size(h)=(bsz, nb_nodes, dim_emb)
        return h, score
    
class Attention(nn.Module):
    def __init__(self, n_hidden):
        super(Attention, self).__init__()
        self.size = 0
        self.batch_size = 0
        self.dim = n_hidden
        
        v  = torch.FloatTensor(n_hidden).cuda()
        self.v  = nn.Parameter(v)
        self.v.data.uniform_(-1/math.sqrt(n_hidden), 1/math.sqrt(n_hidden))
        
        # parameters for pointer attention
        self.Wref = nn.Linear(n_hidden, n_hidden)
        self.Wq = nn.Linear(n_hidden, n_hidden)
    
    
    def forward(self, q, ref):       # query and reference
        self.batch_size = q.size(0)
        self.size = int(ref.size(0) / self.batch_size)
        q = self.Wq(q)     # (B, dim)
        ref = self.Wref(ref)
        ref = ref.view(self.batch_size, self.size, self.dim)  # (B, size, dim)
        
        q_ex = q.unsqueeze(1).repeat(1, self.size, 1) # (B, size, dim)
        # v_view: (B, dim, 1)
        v_view = self.v.unsqueeze(0).expand(self.batch_size, self.dim).unsqueeze(2)
        
        # (B, size, dim) * (B, dim, 1)
        u = torch.bmm(torch.tanh(q_ex + ref), v_view).squeeze(2)
        
        return u, ref
    
class LSTM(nn.Module):
    def __init__(self, n_hidden):
        super(LSTM, self).__init__()
        
        # parameters for input gate
        self.Wxi = nn.Linear(n_hidden, n_hidden)    # W(xt)
        self.Whi = nn.Linear(n_hidden, n_hidden)    # W(ht)
        self.wci = nn.Linear(n_hidden, n_hidden)    # w(ct)
        
        # parameters for forget gate
        self.Wxf = nn.Linear(n_hidden, n_hidden)    # W(xt)
        self.Whf = nn.Linear(n_hidden, n_hidden)    # W(ht)
        self.wcf = nn.Linear(n_hidden, n_hidden)    # w(ct)
        
        # parameters for cell gate
        self.Wxc = nn.Linear(n_hidden, n_hidden)    # W(xt)
        self.Whc = nn.Linear(n_hidden, n_hidden)    # W(ht)
        
        # parameters for forget gate
        self.Wxo = nn.Linear(n_hidden, n_hidden)    # W(xt)
        self.Who = nn.Linear(n_hidden, n_hidden)    # W(ht)
        self.wco = nn.Linear(n_hidden, n_hidden)    # w(ct)
    
    
    def forward(self, x, h, c):       # query and reference
        
        # input gate
        i = torch.sigmoid(self.Wxi(x) + self.Whi(h) + self.wci(c))
        # forget gate
        f = torch.sigmoid(self.Wxf(x) + self.Whf(h) + self.wcf(c))
        # cell gate
        c = f * c + i * torch.tanh(self.Wxc(x) + self.Whc(h))
        # output gate
        o = torch.sigmoid(self.Wxo(x) + self.Who(h) + self.wco(c))
        
        h = o * torch.tanh(c)
        
        return h, c

class HPN(nn.Module):
    def __init__(self, n_feature, n_hidden):

        super(HPN, self).__init__()
        self.city_size = 0
        self.batch_size = 0
        self.dim = n_hidden
        
        # pointer layer
        self.pointer = Attention(n_hidden)
        self.TransPointer = Attention(n_hidden)
        
        # lstm encoder
        self.encoder = LSTM(n_hidden)
        
        # trainable first hidden input
        h0 = torch.FloatTensor(n_hidden)
        c0 = torch.FloatTensor(n_hidden)
    
        self.h0 = nn.Parameter(h0)
        self.c0 = nn.Parameter(c0)
        
        self.h0.data.uniform_(-1/math.sqrt(n_hidden), 1/math.sqrt(n_hidden))
        self.c0.data.uniform_(-1/math.sqrt(n_hidden), 1/math.sqrt(n_hidden))
        
        r1 = torch.ones(1)
        r2 = torch.ones(1)
        r3 = torch.ones(1)
        
        self.r1 = nn.Parameter(r1)
        self.r2 = nn.Parameter(r2)
        self.r3 = nn.Parameter(r3)
        
        # embedding
        self.embedding_x = nn.Linear(n_feature, n_hidden)
        self.embedding_all1 = nn.Linear(n_feature, n_hidden)
        self.embedding_all2 = nn.Linear(n_feature + 1, n_hidden)
        self.Transembedding_all = TransEncoderNet(6, 128, 8, 512, batchnorm=True)
        
        # vector to start decoding 
        self.start_placeholder = nn.Parameter(torch.randn(n_hidden))
        
        # weights for GNN
        self.W1 = nn.Linear(n_hidden, n_hidden)
        self.W2 = nn.Linear(n_hidden, n_hidden)
        self.W3 = nn.Linear(n_hidden, n_hidden)
        
        # aggregation function for GNN
        self.agg_1 = nn.Linear(n_hidden, n_hidden)
        self.agg_2 = nn.Linear(n_hidden, n_hidden)
        self.agg_3 = nn.Linear(n_hidden, n_hidden)
    
    
    def forward(self, Transcontext, x, X_all, mask, h=None, c=None, latent=None):
        '''
        Inputs (B: batch size, size: city size, dim: hidden dimension)
        
        x: current city coordinate (B, 2)
        X_all: all cities' cooridnates (B, size, 2)
        mask: mask visited cities
        h: hidden variable (B, dim)
        c: cell gate (B, dim)
        latent: latent pointer vector from previous layer (B, size, dim)
        
        Outputs
        
        softmax: probability distribution of next city (B, size)
        h: hidden variable (B, dim)
        c: cell gate (B, dim)
        latent_u: latent pointer vector for next layer
        '''
        
        self.batch_size = X_all.size(0)
        self.city_size = X_all.size(1)
        
        # Check if this iteration is the first one
        if h is None or c is None:
            # Letting the placeholder be the first input
            x          = self.start_placeholder
            #  init-embedding for All Cities
            context = self.embedding_all1(X_all)
            # Transormer context 
            Transcontext,_ = self.Transembedding_all(context)
            
            Transcontext = Transcontext.reshape(-1, self.dim) # (B, size, dim)
            
            # =============================
            # handling the cell and the hidden state for the first iteration 
            # =============================
            h0 = self.h0.unsqueeze(0).expand(self.batch_size, self.dim)
            c0 = self.c0.unsqueeze(0).expand(self.batch_size, self.dim)
            h0 = h0.unsqueeze(0).contiguous()
            c0 = c0.unsqueeze(0).contiguous()
            # let h0, c0 be the hidden variable of first turn
            h = h0.squeeze(0)
            c = c0.squeeze(0)
        else:
            # =============================
            # Feature context
            # =============================
            X_all      = torch.cat((torch.cdist(X_all,x.view(self.batch_size,1,2),p=2), X_all - x.unsqueeze(1).repeat(1, self.city_size, 1)), 2)
            # sequential input Embedding 
            x          = self.embedding_x(x)
            #  init-embedding for All Cities
            context = self.embedding_all2(X_all)
            
        # =============================
        # graph neural network encoder
        # =============================
        # Handling contextes's size
        context = context.reshape(-1, self.dim)           # (B, size, dim)
        context = self.r1 * self.W1(context) + (1-self.r1) * F.relu(self.agg_1(context/(self.city_size-1)))
        context = self.r2 * self.W2(context) + (1-self.r2) * F.relu(self.agg_2(context/(self.city_size-1)))
        context = self.r3 * self.W3(context) + (1-self.r3) * F.relu(self.agg_3(context/(self.city_size-1)))
        # LSTM encoder
        h, c = self.encoder(x, h, c)
        
        # =============================
        # Decoding Phase
        # ============================= 
        
        u1, _ = self.pointer(h, context)
        u2 ,_ = self.TransPointer(h,Transcontext)
        u = u1 + u2
        latent_u = u.clone()
        u = 100 * torch.tanh(u) + mask
        return Transcontext,F.softmax(u, dim=1), h, c, latent_u

# Load HPN

In [ ]:
Critic = HPN(n_feature=2, n_hidden=128)
Critic = Critic.to(device)
Critic.eval()

#********************************************# Uncomment these lines to re-start training with saved checkpoint #********************************************#

checkpoint_file = "../input/test-1-128641e3optstep/checkpoint_21-08-09--10-54-32-n50-gpu011Epochs.pkl"
checkpoint = torch.load(checkpoint_file, map_location=device)
epoch_ckpt = checkpoint['epoch'] + 1
tot_time_ckpt = checkpoint['tot_time']
plot_performance_train = checkpoint['plot_performance_train']
plot_performance_baseline = checkpoint['plot_performance_baseline']
Critic.load_state_dict(checkpoint['model_baseline'])

print('Re-start training with saved checkpoint file={:s}\n  Checkpoint at epoch= {:d} and time={:.3f}min\n'.format(checkpoint_file,epoch_ckpt-1,tot_time_ckpt/60))
del checkpoint

#*********************************************# Uncomment these lines to re-start training with saved checkpoint #********************************************#

# Test Batch

## HPN Without 2opt

In [ ]:
B = 50

total_tour_len = 0
n_test = int(1000/B)

# there are 1000 test data
size = 250
Z = torch.rand(n_test, B, size, 2).clone()


for m in tqdm_notebook(range(n_test)):
    X = Z[m].cuda()
    
    mask = torch.zeros(B,size).cuda()
    
    R = 0
    solution = []
    reward = 0
    
    Y = X.view(B, size, 2)           # to the same batch size
    x = Y[:,0,:]
    h = None
    c = None
    Transcontext = None
    for k in range(size):
        Transcontext,output, h, c, _ = Critic(Transcontext,x=x, X_all=X, h=h, c=c, mask=mask)
        
        idx = torch.argmax(output, dim=1)
        
        x = Y[[i for i in range(B)], idx.data]
        solution.append(x.cpu().numpy())
        
        mask[[i for i in range(B)], idx.data] += -np.inf
    
    solution.append(solution[0])
    graph = np.array(solution)
    route = [x for x in range(size)] + [0]


    for b in range(B):
        best = route.copy()
        # begin 2-opt
        graph_ = graph[:,b,:].copy()
            
        dmatrix = distance.cdist(graph_, graph_, 'euclidean')
        improved = True
        '''
        while improved:
            improved = False
            
            for i in range(size):
                for j in range(i+2, size+1):
                    
                    old_dist = dmatrix[best[i],best[i+1]] + dmatrix[best[j], best[j-1]]
                    new_dist = dmatrix[best[j],best[i+1]] + dmatrix[best[i], best[j-1]]
                    
                    # new_dist = 1000
                    if new_dist < old_dist:
                        best[i+1:j] = best[j-1:i:-1]
                        # print(opt_tour)
                        improved = True
        '''
        new_tour_len = 0
        for k in range(size):
            new_tour_len += dmatrix[best[k], best[k+1]]

        
        total_tour_len += new_tour_len
    print("sample:{}, new route length:{}".format(m, new_tour_len))
    
print('total length:', total_tour_len/1000)

## HPN With 2opt

In [ ]:
B = 50

total_tour_len = 0
n_test = int(1000/B)

# there are 1000 test data
size = 250
Z = torch.rand(n_test, B, size, 2).clone()


for m in tqdm_notebook(range(n_test)):
    X = Z[m].cuda()
    
    mask = torch.zeros(B,size).cuda()
    
    R = 0
    solution = []
    reward = 0
    
    Y = X.view(B, size, 2)           # to the same batch size
    x = Y[:,0,:]
    h = None
    c = None
    Transcontext = None
    for k in range(size):
        Transcontext,output, h, c, _ = Critic(Transcontext,x=x, X_all=X, h=h, c=c, mask=mask)
        
        idx = torch.argmax(output, dim=1)
        
        x = Y[[i for i in range(B)], idx.data]
        solution.append(x.cpu().numpy())
        
        mask[[i for i in range(B)], idx.data] += -np.inf
    
    solution.append(solution[0])
    graph = np.array(solution)
    route = [x for x in range(size)] + [0]


    for b in range(B):
        best = route.copy()
        # begin 2-opt
        graph_ = graph[:,b,:].copy()
            
        dmatrix = distance.cdist(graph_, graph_, 'euclidean')
        improved = True
        
        while improved:
            improved = False
            
            for i in range(size):
                for j in range(i+2, size+1):
                    
                    old_dist = dmatrix[best[i],best[i+1]] + dmatrix[best[j], best[j-1]]
                    new_dist = dmatrix[best[j],best[i+1]] + dmatrix[best[i], best[j-1]]
                    
                    # new_dist = 1000
                    if new_dist < old_dist:
                        best[i+1:j] = best[j-1:i:-1]
                        # print(opt_tour)
                        improved = True
        
        new_tour_len = 0
        for k in range(size):
            new_tour_len += dmatrix[best[k], best[k+1]]

        
        total_tour_len += new_tour_len
    print("sample:{}, new route length:{}".format(m, new_tour_len))
    
print('total length:', total_tour_len/1000)


# Data pre-processing For benchmark Instances 

In [ ]:
# Data path
data_path='../input/pa561data/pa561.tsp'
data = np.loadtxt(data_path,dtype='float32')

# Delete the labeling col from the file
data = np.delete(data, 0, axis=1)
size = data.shape[0]
test_data = torch.from_numpy(data)
data2=torch.zeros(size,2)

# normalize data 
min = torch.min(test_data,0)
max = torch.max(test_data,0)
data2[:,0] = (test_data[:,0] - min[0][0]) / (max[0][0]-min[0][0])
data2[:,1] = (test_data[:,1] - min[0][1]) / (max[0][1]-min[0][1])

Z = data2.view(1,1, size, 2).to(device)
Z.required_grad = False

## Without 2opt

In [ ]:
start = time.time()
total_tour_len = 0
n_test = 1
B = 1
with torch.no_grad():
    for m in tqdm.notebook.tqdm(range(n_test)):
        X = Z[m].cuda()
        mask = torch.zeros(B,size).cuda()
        R = 0
        solution = []
        solution_real = []
        reward = 0
        Y = X.view(B, size, 2)           # to the same batch size
        test_data = test_data.view(B,size,2)
        x = Y[:,0,:]
        h = None
        c = None
        Transcontext = None
        for k in range(size):
            Transcontext,output, h, c, _ = Critic(Transcontext,x=x, X_all=X, h=h, c=c, mask=mask)
            idx = torch.argmax(output, dim=1)
            x      = Y[[i for i in range(B)], idx.data]
            x_real = test_data[[i for i in range(B)], idx.data]
            solution.append(x.cpu().numpy())
            solution_real.append(x_real.cpu().numpy())
            mask[[i for i in range(B)], idx.data] += -np.inf
        solution.append(solution[0])
        graph = np.array(solution)
        graph_real = np.array(solution_real)
        route = [x for x in range(size)] + [0]
        for b in range(B):
            best = route.copy()
            # begin 2-opt
            graph_ = graph[:,b,:].copy()
            dmatrix = distance.cdist(graph_, graph_, 'euclidean')
            improved = True
            """
            while improved:
                improved = False
                for i in range(size):
                    for j in range(i+2, size+1):
                        old_dist = dmatrix[best[i],best[i+1]] + dmatrix[best[j], best[j-1]]
                        new_dist = dmatrix[best[j],best[i+1]] + dmatrix[best[i], best[j-1]]
                        if new_dist < old_dist:
                            best[i+1:j] = best[j-1:i:-1]
                            improved = True
                            
            """ 
            new_tour_len = 0
            for k in range(size):
                new_tour_len += dmatrix[best[k], best[k+1]]
            total_tour_len += new_tour_len
        print("sample:{}, new route length:{}".format(m, new_tour_len))
    print('total length:', total_tour_len)
    print('tot time',time.time() - start)

## With 2opt

In [ ]:
start = time.time()
total_tour_len = 0
with torch.no_grad():
    for m in tqdm.notebook.tqdm(range(n_test)):
        X = Z[m].cuda()
        mask = torch.zeros(B,size).cuda()
        R = 0
        solution = []
        solution_real = []
        reward = 0
        Y = X.view(B, size, 2)           # to the same batch size
        test_data = test_data.view(B,size,2)
        x = Y[:,0,:]
        h = None
        c = None
        Transcontext = None
        for k in range(size):
            Transcontext,output, h, c, _ = Critic(Transcontext,x=x, X_all=X, h=h, c=c, mask=mask)
            idx = torch.argmax(output, dim=1)
            x      = Y[[i for i in range(B)], idx.data]
            x_real = test_data[[i for i in range(B)], idx.data]
            solution.append(x.cpu().numpy())
            solution_real.append(x_real.cpu().numpy())
            mask[[i for i in range(B)], idx.data] += -np.inf
        solution.append(solution[0])
        graph = np.array(solution)
        graph_real = np.array(solution_real)
        route = [x for x in range(size)] + [0]
        for b in range(B):
            best = route.copy()
            # begin 2-opt
            graph_ = graph[:,b,:].copy()
            dmatrix = distance.cdist(graph_, graph_, 'euclidean')
            improved = True
            while improved:
                improved = False
                for i in range(size):
                    for j in range(i+2, size+1):
                        old_dist = dmatrix[best[i],best[i+1]] + dmatrix[best[j], best[j-1]]
                        new_dist = dmatrix[best[j],best[i+1]] + dmatrix[best[i], best[j-1]]
                        if new_dist < old_dist:
                            best[i+1:j] = best[j-1:i:-1]
                            improved = True
            new_tour_len = 0
            for k in range(size):
                new_tour_len += dmatrix[best[k], best[k+1]]
            total_tour_len += new_tour_len
        print("sample:{}, new route length:{}".format(m, new_tour_len))
    print('total length:', total_tour_len)
    print('tot time',time.time() - start)

# Unnormalized distance

In [ ]:
graph_real = torch.tensor(graph_real,dtype = torch.float64).squeeze(1).unsqueeze(0)
L = compute_tour_length(graph_real,torch.tensor(best).unsqueeze(0))
print(L)

# Simple visualization

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

TSP = np.zeros([size+1,2],dtype=np.float32)
for i in range(TSP.shape[0]):
    idx = best[i]
    TSP[i] = graph_[idx,:]

fig = plt.figure(figsize=(10,10))
plt.scatter(TSP[:,0],TSP[:,1],s = 10)
plt.plot(TSP[:,0],TSP[:,1])
#Saving the plot as an image
fig.savefig('ei8246.jpg', bbox_inches='tight', dpi=150)